In [54]:
#Creating Products table
import pandas as pd
import mysql.connector
import numpy as np
try:
    encoding = 'ISO-8859-1'
    #Reading data from CSV file
    df=pd.read_csv(r"C:\Users\Priyanka\OneDrive\Desktop\practice\DataSpark\Products.csv",encoding=encoding)
    df.isna().sum()#Verifying any null values in data
    finalproductslist=[]
    for i in df.index:
        product=[]
        for j in df.loc[i].values:
            if "," in str(j):
                j=str(j).replace(",","")
            if "$" in str(j):
                j=float(str(j).replace("$",""))#Replacing special characters
            if "&" in str(j):
                j=str(j).replace("&","")
            product.append(j) 
            product = [int(x) if isinstance(x, (np.int64, np.int32)) else x for x in product]
            product = [float(x) if isinstance(x, (np.float64, np.float32)) else x for x in product]
        finalproductslist.append(tuple(product))
    #Connecting to the database
    con=mysql.connector.connect(
                host="localhost",
                user="root",
                password="23051991",
                database="MDE92"
            )
    cursor=con.cursor()
    #Creating Products table
    query="""create table if not exists mde92.Products (ProductKey int primary key,
                ProductName varchar(255),
                Brand varchar(20),
                Color varchar(20),
                UnitCostUSD float,
                UnitPriceUSD float,
                SubcategoryKey int,
                Subcategory varchar(255),
                CategoryKey int,
                Category varchar(50))"""
    cursor.execute(query)
    #Inserting values into Products table
    query="insert into mde92.products values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(query,finalproductslist)
    con.commit()
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 1062 (23000): Duplicate entry '1' for key 'products.PRIMARY'


In [58]:
#Creating Exchange Rates table
import pandas as pd
import numpy as np
import mysql.connector
try:
    encoding = 'ISO-8859-1'
    #Reading data from CSV file
    df=pd.read_csv(r"C:\Users\Priyanka\OneDrive\Desktop\practice\DataSpark\Exchange_Rates.csv",encoding=encoding)
    #Saving date in datetime form matching to sql datetime format
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce')
    df.isna().sum()#Verifying any null values in table
    finalexchangelist=[]
    batch_size=1000
    for i in df.index:
        exchange=[]
        row = df.loc[i]
        er_id=i
        exchange_date = row['Date'].strftime('%Y-%m-%d')
        currency = row['Currency']
        exchange_rate = row['Exchange']
        exchange.extend([i,exchange_date, currency, exchange_rate])
        #Sql accept only float values
        exchange = [float(x) if isinstance(x, (np.float64, np.float32)) else x for x in exchange]
        finalexchangelist.append(tuple(exchange))
    #Connecting to the database
    con=mysql.connector.connect(
                    host="localhost",
                    user="root",
                    password="23051991",
                    database="MDE92"
                )
    cursor=con.cursor()
    #Creating Exchange Rates table
    query=f"""create table if not exists mde92.Exchange_Rates (ERid int primary key,
            ExchangeDate datetime,currency varchar(50),ExchangeRate float);"""
    cursor.execute(query)
    #Inserting values into table
    query="INSERT INTO mde92.Exchange_Rates (ERid,ExchangeDate, currency, ExchangeRate) VALUES (%s,%s,%s,%s)"
    for i in range(0, len(finalexchangelisthangelist), batch_size):
        batch = finalexchangelist[i:i+batch_size]
        cursor.executemany(query, batch)
        con.commit()
except Exception as e:
    print(f"An error occurred: {e}")

In [66]:
#Creating Customers table
import pandas as pd
import numpy as np
import mysql.connector
try:
    encoding = 'ISO-8859-1'
    #Reading data from CSV file
    df=pd.read_csv(r"C:\Users\Priyanka\OneDrive\Desktop\practice\DataSpark\Customers.csv",encoding=encoding)
    df.isna().sum() #To check any null values in the dataa
    df["State Code"]=df["State Code"].ffill() #Filling the values with ffill
    #Converting date to datetime format matches to sql datetime format
    df['Birthday'] = pd.to_datetime(df['Birthday'], format='%m/%d/%Y', errors='coerce')
    finalcustomerlist=[]
    for i in df.index:
        customer=[]
        row=df.loc[i]
        customer_id=i
        customerkey=row['CustomerKey']
        gender=row['Gender']
        name=row['Name']
        city=row['City']
        statecode=row['State Code']
        state=row['State']
        zipcode=row['Zip Code']
        country=row['Country']
        continent=row['Continent']
        birthday=row['Birthday'].strftime('%Y-%m-%d')
        customer.extend([i,customerkey,gender,name,city,statecode,state,zipcode,country,continent,birthday])
        customer = [int(x) if isinstance(x, (np.int64, np.int32)) else x for x in customer]
        finalcustomerlist.append(tuple(customer))
    #Connecting to the database
    con=mysql.connector.connect(
                        host="localhost",
                        user="root",
                        password="23051991",
                        database="MDE92"
                    )
    cursor=con.cursor()
    #Creating customers table
    query="""create table if not exists mde92.customers (customer_id int unique not null,
             customer_key int primary key,gender varchar(20),name varchar(255),city varchar(255),state_code varchar(255),
             state varchar(255),zip_code varchar(25),country varchar(255),continent varchar(255),birthday datetime);"""
    cursor.execute(query)
    #Inserting values into customers table
    query="""insert into mde92.customers(customer_id,customer_key,gender,name,city,state_code,state,zip_code,country,continent,birthday) 
             values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    #Insert more than 1 row at once
    cursor.executemany(query,finalcustomerlist)
    con.commit()
except Exception as e:
    print(f"An error occured:{e}")

In [64]:
#Creating Stores table
import pandas as pd
import numpy as np
import mysql.connector
try:
    encoding = 'ISO-8859-1'
    #Reading data CSV file
    df=pd.read_csv(r"C:\Users\Priyanka\OneDrive\Desktop\practice\DataSpark\Stores.csv",encoding=encoding)
    df.isna().sum()#Verifying any null values in the data
    df["Square Meters"]=df["Square Meters"].ffill()#Filling null values with front data
    df.isna().sum()
    #Converting datetime matches to sql datetime format
    df['Open Date'] = pd.to_datetime(df['Open Date'], format='%m/%d/%Y', errors='coerce')
    finalstorelist=[]
    for i in df.index:
        store=[]
        row=df.loc[i]
        store_key=row['StoreKey']
        country=row['Country']
        state=row['State']
        square_meters=row['Square Meters']
        date=row['Open Date'].strftime('%Y-%m-%d')
        store.extend([store_key,country,state,square_meters,date])
        store = [float(x) if isinstance(x, (np.float64, np.float32)) else x for x in store]
        store = [int(x) if isinstance(x, (np.int64, np.int32)) else x for x in store]
        finalstorelist.append(tuple(store))
    #Connecting to database
    con=mysql.connector.connect(
                        host="localhost",
                        user="root",
                        password="23051991",
                        database="MDE92"
                    )
    cursor=con.cursor()
    #Creating Stores table
    query="""create table if not exists mde92.stores(store_key int primary key,
           country varchar(255),state varchar(255),square_meters float,open_date datetime);"""
    cursor.execute(query)
    #Inserting values into stores table
    query="""insert into mde92.stores(store_key,country,state,square_meters,open_date) values(%s,%s,%s,%s,%s)"""
    cursor.executemany(query,finalstorelist)
    con.commit()
except Exception as e:
    print(f"An error occured:{e}")


In [78]:
import pandas as pd
import numpy as np
import mysql.connector
try:
    encoding = 'ISO-8859-1'
    #Reading data form CSV file
    df=pd.read_csv(r"C:\Users\Priyanka\OneDrive\Desktop\practice\DataSpark\Sales.csv",encoding=encoding)
    df.isna().sum()
    df["Delivery Date"]=df["Delivery Date"].ffill()#Filling null values with front values
    df.isna().sum()#Verifying any null values in the data
    df=df.dropna()#Dropping rows with null values
    #Converting date matches to sql datetime format
    df['Order Date'] = pd.to_datetime(df['Order Date']).dt.date
    df['Delivery Date'] = pd.to_datetime(df['Delivery Date']).dt.date
    finalsaleslist=[]
    for i in df.index:
        sale=[]
        row=df.loc[i]
        orderno=int(row['Order Number'])
        lineitem=int(row['Line Item'])
        orderdate=row['Order Date'].strftime('%Y-%m-%d')
        deliverydate=row['Delivery Date'].strftime('%Y-%m-%d')
        customerkey=int(row['CustomerKey'])
        storekey=int(row['StoreKey'])
        productkey=int(row['ProductKey'])
        quantity=int(row['Quantity'])
        currencycode=row['Currency Code']
        finalsaleslist.append(tuple([orderno,lineitem,orderdate,deliverydate,customerkey,storekey,productkey,quantity,currencycode]))
    #Connecting to database
    con=mysql.connector.connect(
                        host="localhost",
                        user="root",
                        password="23051991",
                        database="MDE92"
                    )
    cursor=con.cursor()
    #Creating sales table
    query="""create table if not exists mde92.sales(order_number int primary key,
            line_item int,
            order_date datetime,
            delivery_date datetime,	
            customer_id int,foreign key(customer_id) references mde92.customers(customer_key),
            store_id int,foreign key(store_id) references mde92.stores(store_key),
            product_id int,foreign key(product_id) references mde92.products(productkey),
            quantity int,
            currency_code varchar(255));"""
    cursor.execute(query)
    #Inserting values into sales table
    query="""insert into mde92.sales(order_number,line_item,order_date,delivery_date,customer_id,store_id,product_id,
             quantity,currency_code) 
             values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cursor.executemany(query,finalsaleslist)
    con.commit()
except Exception as e:
    print(f"An error occured:{e}")

